In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
import xgboost as xgb
from sklearn.metrics import accuracy_score
from transformers.dummies_encoder import DummiesEncoder
from transformers.item_selector import ItemSelector
from transformers.morphology_extractor import MorphologyExtractor
from transformers.dict_class_transformer import DictClassTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.datasets import dump_svmlight_file
from transformers.multi_label_encoder import MultiLabelEncoder
from transformers.reshape_2d import Reshape2D
from transformers.pandas_union import PandasUnion
from transformers.pandas_shift import PandasShift
from transformers.string_splitter import StringSplitter
from loading import load_train
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot
import seaborn as sns

%matplotlib inline

#Prepare data

In [2]:
df = load_train(['before', 'class']).sample(1000, random_state=2017)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 4656679 to 7180668
Data columns (total 2 columns):
class     1000 non-null object
before    1000 non-null object
dtypes: object(2)
memory usage: 23.4+ KB


In [3]:
pipeline = Pipeline([
    # ('features', FeatureUnion([
    ('select', ItemSelector('before')),
    ('features', PandasUnion([
        ('char', Pipeline([
            ('split', StringSplitter(10))
        ])),
        ('ctx', Pipeline([
            ('extract', MorphologyExtractor()),
            ('one_hot', DummiesEncoder())
        ])),
        ('char_prev', Pipeline([
            ('shift', PandasShift(1)),
            ('split', StringSplitter(5))
        ])),
        ('ctx_prev', Pipeline([
            ('shift', PandasShift(1)),
            ('extract', MorphologyExtractor()),
            ('one_hot', DummiesEncoder())
        ])),
        ('char_next', Pipeline([
            ('shift', PandasShift(-1)),
            ('split', StringSplitter(5))
        ])),
        ('ctx_next', Pipeline([
            ('shift', PandasShift(-1)),
            ('extract', MorphologyExtractor()),
            ('one_hot', DummiesEncoder())
        ])),
    ])),
])

x_data = pipeline.fit_transform(df.drop(['class'], axis=1))
print(x_data.shape)

y_data = pd.factorize(df['class'])
labels = y_data[1]
y_data = y_data[0]
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=2017)

MorphologyExtractor transform:   0%|          | 0/1000 [00:00<?, ?it/s]

MorphologyExtractor transform: 100%|██████████| 1000/1000 [00:00<00:00, 14272.85it/s]


MorphologyExtractor transform:   0%|          | 0/1000 [00:00<?, ?it/s]

MorphologyExtractor transform: 100%|██████████| 1000/1000 [00:00<00:00, 13133.22it/s]


MorphologyExtractor transform:   0%|          | 0/1000 [00:00<?, ?it/s]

...

MorphologyExtractor transform:   0%|          | 0/1000 [00:00<?, ?it/s]

MorphologyExtractor transform: 100%|██████████| 1000/1000 [00:00<00:00, 26243.27it/s]

MorphologyExtractor transform:   0%|          | 0/1000 [00:00<?, ?it/s]

MorphologyExtractor transform: 100%|██████████| 1000/1000 [00:00<00:00, 34716.17it/s]

(1000, 155)


In [69]:
dump_svmlight_file(x_train, y_train, 'models/class.txt.train')
dump_svmlight_file(x_test, y_test, 'models/class.txt.test')

#Search number of estimators by early stopping

In [4]:
dtrain = xgb.DMatrix('models/class.txt.train#dtrain.cache')
# dtest = xgb.DMatrix('models/class.txt.test#dtest.cache')

In [13]:
num_classes = len(set(dtrain.get_label()))

199

In [7]:
xgb_param = {
    'learning_rate': 0.1,
    'num_boost_round': 1000,
    'max_depth': 5,
    'min_child_weight': 1,
    'gamma': 0,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'multi:softmax',
    'num_class': num_classes,
    'nthread': 4,
    'scale_pos_weight': 1,
    'seed': 2017}
cvresult = xgb.cv(xgb_param, dtrain, num_boost_round=xgb_param['num_boost_round'], 
                  stratified=True, nfold=10,
                  metrics=['merror'], early_stopping_rounds=100, verbose_eval=True)
print('Best num_boost_round value', cvresult.shape[0])

pyplot.errorbar(cvresult.index, cvresult['train-merror-mean'], 
                yerr=cvresult['train-merror-std'], ecolor='r')
pyplot.errorbar(cvresult.index, cvresult['test-merror-mean'], 
                yerr=cvresult['test-merror-std'], ecolor='g')
pyplot.title("XGBoost num_boost_round vs MError")
pyplot.xlabel('num_boost_round')
pyplot.ylabel('MError')
pyplot.savefig('num_boost_round.png')

KeyboardInterrupt: 

In [ ]:
watchlist = [(dtest, 'test'), (dtrain, 'train')]
param = {'objective': 'multi:softmax',
         'eta': '0.3',
         'max_depth': 5,
         'silent': 1,
         'nthread': -1,
         'num_class': len(labels),
         'eval_metric': 'merror',
         'seed': '2017'}
model = xgb.train(param, dtrain, num_boost_round=cvresult.shape[0], evals=watchlist, 
                  early_stopping_rounds=100, verbose_eval=True)

predicted_val = model.predict(dtrain)
print(f'pipeline val error {1.0-accuracy_score(y_train, predicted_val)}', flush=True)
predicted = model.predict(dtest)
print(f'pipeline test error {1.0-accuracy_score(y_test, predicted)}', flush=True)

plt.rcParams['font.size'] = 8
feat_imp = pd.Series(model.get_fscore()).sort_values(ascending=False)
print(feat_imp.index)
print(x_data.columns[~x_data.columns.isin(feat_imp.index)])
feat_imp.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')
plt.tight_layout()
plt.show()

#CV search

In [19]:
model = xgb.XGBClassifier()
n_estimators = range(100, 1000, 100)
param_grid = dict(n_estimators=n_estimators)
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=2017)
grid_search = GridSearchCV(model, param_grid, scoring="accuracy", n_jobs=-1, cv=kfold, verbose=10)
grid_result = grid_search.fit(x_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

pyplot.errorbar(n_estimators, means, yerr=stds)
pyplot.title("XGBoost n_estimators vs Accuracy")
pyplot.xlabel('n_estimators')
pyplot.ylabel('Accuracy')
pyplot.savefig('n_estimators.png')

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[CV] n_estimators=100 ................................................


[CV] n_estimators=100 ................................................


[CV] n_estimators=100 ................................................


[CV] n_estimators=200 ................................................


KeyboardInterrupt: 

In [ ]:
model = XGBClassifier()
max_depth = range(1, 11, 2)
print(max_depth)
param_grid = dict(max_depth=max_depth)
model = xgb.XGBClassifier()
n_estimators = range(100, 1000, 100)
param_grid = dict(n_estimators=n_estimators)
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=2017)
grid_search = GridSearchCV(model, param_grid, scoring="accuracy", n_jobs=-1, cv=kfold, verbose=10)
grid_result = grid_search.fit(x_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

pyplot.errorbar(n_estimators, means, yerr=stds)
pyplot.title("XGBoost n_estimators vs Accuracy")
pyplot.xlabel('n_estimators')
pyplot.ylabel('Accuracy')
pyplot.savefig('n_estimators.png')

MorphologyExtractor transform:   0%|          | 0/100000 [00:00<?, ?it/s]

MorphologyExtractor transform:   1%|▏         | 1318/100000 [00:00<00:07, 13173.10it/s]

MorphologyExtractor transform:   2%|▏         | 2363/100000 [00:00<00:08, 11658.19it/s]

MorphologyExtractor transform:   4%|▍         | 3767/100000 [00:00<00:07, 12443.39it/s]

MorphologyExtractor transform:   5%|▍         | 4596/100000 [00:00<00:10, 9444.87it/s] 

...

MorphologyExtractor transform:  92%|█████████▏| 91538/100000 [00:04<00:00, 21332.56it/s]

MorphologyExtractor transform:  94%|█████████▍| 94191/100000 [00:04<00:00, 21450.80it/s]

MorphologyExtractor transform:  97%|█████████▋| 96796/100000 [00:04<00:00, 21552.54it/s]

MorphologyExtractor transform:  99%|█████████▉| 99399/100000 [00:04<00:00, 21650.07it/s]

MorphologyExtractor transform: 100%|██████████| 100000/100000 [00:04<00:00, 21657.18it/s]